# Cleaning up Grants Data
----

There are three sheets in the TIRCP workbook. However, it seems like the summary reports mostly rely on two: one called project tracking and one called allocation tracking, so I am only loading those in.


In [1]:
import pandas as pd
import math
!pip install openpyxl
pd.options.display.max_rows = 100

In [2]:
GCS_FILE_PATH = "gs://calitp-analytics-data/data-analyses/tircp/"
FILE_NAME1 = "Raw_Project_Tracking_Sheet.xlsx"
project = pd.read_excel(f"{GCS_FILE_PATH}{FILE_NAME1}")
FILE_NAME2 = "Allocation_Agreement.xlsx"
allocation = pd.read_excel(f"{GCS_FILE_PATH}{FILE_NAME2}")

In [3]:
#cleaning up spaces in columns
project.columns = project.columns.str.strip().str.replace(' ', '_')
allocation.columns = allocation.columns.str.strip().str.replace(' ', '_')

### Note: Third party award date is called CON Contract Award Date in TIRCP SAR Attachment 

In [4]:
#subsetting for only columns of interest
df_project = project[['Award_Year', 'Project_#','Local_Agency','Project_Title','PPNO',
                      'Key_Project_Elements','TIRCP_Award_Amount_($)','Allocated_Amount','Expended_Amount']]

In [5]:
#subsetting for only columns of interest
df_allocation = allocation[['Award_Year','Award_Recipient', 'Implementing_Agency', 'PPNO','Phase', 'Allocation_Date','Completion_Date','3rd_Party_Award_Date','Components']]

In [6]:
#strip spaces in columns
df_project.columns = df_project.columns.map(lambda x: x.strip())
df_allocation.columns = df_allocation.columns.map(lambda x: x.strip())

# Cleaning Allocation Sheet 

## Cleaning up PPNO, can only be 5 characters.

In [7]:
#remove the extra characters in PPNO in allocation to match the PPNO in project data frame bc there should only be five characters and numbers in each PPNO value
df_allocation = df_allocation.assign(
    PPNO_New = df_allocation['PPNO'].str.slice(start=0, stop=5)
)

In [8]:
df_allocation.shape

(307, 10)

In [9]:
#test = df_allocation[(df_allocation.Award_Recipient.str.contains("Metropolitan", case= False))]

In [10]:
#CSV with PPNO & Award Recipients
FILE_NAME2 = "Allocation_PPNO_Crosswalk.csv"
allocation_ppno = pd.read_csv(f"{GCS_FILE_PATH}{FILE_NAME2}")

In [11]:
allocation_ppno

,Award_Year,PPNO_New2,Award_Recipient
0,2020,CP065,Los Angeles County Metropolitan Transportation...
1,2020,CP066,Los Angeles-San Diego-San Luis Obispo Rail Cor...
2,2016,1230,San Bernardino County Transportation Authority...
3,2018,1155,Transportation Agency for Monterey County


In [12]:
#Filtering out for 2021, since that entry is blank
df_allocation = df_allocation.query("Award_Year != 2021")

In [13]:
#Merge in Crosswalk 
df_allocation = pd.merge(df_allocation, allocation_ppno, on = ["Award_Year", "Award_Recipient"], how = "left")

In [14]:
type(df_allocation.iloc[0].PPNO_New2)

float

In [15]:
#some values in PPNO and PPNO_New2 are strings, some are floats...so have to convert PPNO New 2 to strings
df_allocation.PPNO_New = df_allocation.apply(lambda x: x.PPNO_New if (str(x.PPNO_New2) == 'nan') else x.PPNO_New2, axis=1)

In [16]:
#drop old column
df_allocation = df_allocation.drop(['PPNO'], axis=1)

In [17]:
#renaming columns to something neater
df_allocation = df_allocation.rename(columns = {'PPNO_New':'PPNO', '3rd_Party_Award_Date':'Third_Party_Award_Date'})

## Cleaning up completion, allocation, & 3rd Party dates 

In [18]:
#cleaning up allocation dates
df_allocation.Allocation_Date.unique().tolist()

[datetime.datetime(2015, 10, 22, 0, 0),
 datetime.datetime(2016, 5, 19, 0, 0),
 datetime.datetime(2016, 6, 30, 0, 0),
 datetime.datetime(2015, 12, 10, 0, 0),
 datetime.datetime(2015, 8, 27, 0, 0),
 datetime.datetime(2016, 1, 21, 0, 0),
 datetime.datetime(2017, 6, 29, 0, 0),
 datetime.datetime(2016, 10, 20, 0, 0),
 datetime.datetime(2017, 8, 17, 0, 0),
 datetime.datetime(2018, 1, 31, 0, 0),
 datetime.datetime(2017, 1, 19, 0, 0),
 datetime.datetime(2016, 3, 17, 0, 0),
 datetime.datetime(2017, 3, 16, 0, 0),
 datetime.datetime(2017, 5, 17, 0, 0),
 datetime.datetime(2018, 8, 16, 0, 0),
 'TBD',
 datetime.datetime(2021, 6, 24, 0, 0),
 datetime.datetime(2016, 12, 8, 0, 0),
 datetime.datetime(2020, 6, 25, 0, 0),
 datetime.datetime(2019, 12, 5, 0, 0),
 datetime.datetime(2018, 10, 18, 0, 0),
 datetime.datetime(2021, 1, 28, 0, 0),
 nan,
 datetime.datetime(2018, 2, 1, 0, 0),
 datetime.datetime(2018, 5, 17, 0, 0),
 'FY 26/27',
 datetime.datetime(2017, 5, 18, 0, 0),
 datetime.datetime(2018, 6, 28, 0,

In [19]:
#Had to change FY to an actual date 
df_allocation["Allocation_Date"].replace({"FY 26/27": "2026-12-31", "08/12//20": '2020-08-12 00:00:00', 'FY 21/22': '2021-12-31',
                                         'FY 22/23': '2022-12-31','FY 20/21': '2020-12-31', 'FY 23/24': '2023-12-31','FY 24/25': '2024-12-31','FY 25/26': '2025-12-31'}, inplace =True)

In [20]:
#clean up columns in a loop
for i in ["Allocation_Date", "Third_Party_Award_Date", "Completion_Date"]:
    df_allocation[i] = df_allocation[i].replace('/', '-', regex = True).replace('Complete', '', regex = True).replace('\n', '', regex=True).replace('Pending','TBD',regex= True).fillna('TBD')

In [21]:
df_allocation.Completion_Date.unique().tolist()

[datetime.datetime(2022, 3, 30, 0, 0),
 '6-1-2019',
 datetime.datetime(2021, 6, 30, 0, 0),
 datetime.datetime(2018, 9, 30, 0, 0),
 '2-11-2018',
 '6-30-2020',
 datetime.datetime(2020, 9, 30, 0, 0),
 ' 6-30-2018',
 '6-29-2020',
 '11-1-2019',
 ' 12-10-2018',
 ' 11-13-2019',
 '3-30-2020',
 datetime.datetime(2022, 9, 30, 0, 0),
 datetime.datetime(2021, 12, 30, 0, 0),
 datetime.datetime(2021, 9, 30, 0, 0),
 '5-16-2020',
 datetime.datetime(2024, 6, 30, 0, 0),
 'TBD',
 'June 24. 2024',
 datetime.datetime(2022, 12, 30, 0, 0),
 datetime.datetime(2024, 6, 24, 0, 0),
 '11-21-20247-30-2025 (Q4)',
 datetime.datetime(2022, 6, 30, 0, 0),
 datetime.datetime(2019, 5, 21, 0, 0),
 datetime.datetime(2024, 7, 25, 0, 0),
 datetime.datetime(2021, 12, 31, 0, 0),
 datetime.datetime(2024, 1, 28, 0, 0),
 datetime.datetime(2022, 10, 31, 0, 0),
 datetime.datetime(2022, 1, 16, 0, 0),
 datetime.datetime(2018, 2, 1, 0, 0),
 datetime.datetime(2022, 8, 22, 0, 0),
 datetime.datetime(2022, 7, 31, 0, 0),
 '5-7-2020',
 date

In [22]:
df_allocation.head(50)

,Award_Year,Award_Recipient,Implementing_Agency,Phase,Allocation_Date,Completion_Date,Third_Party_Award_Date,Components,PPNO,PPNO_New2
0,2015,Antelope Valley Transit Authority,Antelope Valley Transit Authority,CONST,2015-10-22 00:00:00,2022-03-30 00:00:00,2016-03-14 00:00:00,Purchase 13 60-foot articulated BRT buses and ...,CP005,NaN
1,2015,Capitol Corridor Joint Powers Authority,Capitol Corridor Joint Powers Authority,CONST,2016-05-19 00:00:00,6-1-2019,2016-06-01 00:00:00,Track and curve improvements between San Jose ...,CP012,NaN
2,2015,Los Angeles County Metropolitan Transportation...,Los Angeles County Metropolitan Transportation...,CONST,2016-06-30 00:00:00,2021-06-30 00:00:00,2017-06-28 00:00:00,Replace Blue Line signal system; install new t...,CP015,NaN
3,2015,Los Angeles-San Diego-San Luis Obispo Rail Cor...,Los Angeles-San Diego-San Luis Obispo Rail Cor...,CONST,2015-12-10 00:00:00,2021-06-30 00:00:00,2016-06-01 00:00:00,LOSSAN and 12 transit agencies from San Luis O...,CP007,NaN
4,2015,Monterey-Salinas Transit,Monterey-Salinas Transit,CONST,2016-05-19 00:00:00,2018-09-30 00:00:00,2016-11-03 00:00:00,Renovation and expansion of the Monterey maint...,CP013,NaN
5,2015,Orange County Transportation Authority (OCTA),Orange County Transportation Authority (OCTA),CONST,2015-10-22 00:00:00,2-11-2018,2015-11-30 00:00:00,Purchase five 40-foot CNG buses for BRT Route ...,CP004,NaN
6,2015,Sacramento Regional Transit District,Sacramento Regional Transit District,CONST,2015-08-27 00:00:00,6-30-2020,2015-10-21 00:00:00,Refurbishment of Seven UTDC Light Rail Vehicle...,CP001,NaN
7,2015,San Diego Association of Governments (SANDAG),San Diego Association of Governments (SANDAG),CONST,2015-08-27 00:00:00,2020-09-30 00:00:00,2016-01-08 00:00:00,Bus Rapid Transit Route between Downtown San D...,CP003,NaN
8,2015,San Diego Metropolitan Transit System (MTS),San Diego Metropolitan Transit System (MTS),PS&E,2016-01-21 00:00:00,6-30-2018,-,Courthouse Trolley Station (Design),CP008,NaN
9,2015,San Diego Metropolitan Transit System (MTS),San Diego Metropolitan Transit System (MTS),CONST,2017-06-29 00:00:00,6-29-2020,2017-05-30 00:00:00,Courthouse Trolley Station (Construction),CP008,NaN


#cleaning up completion dates
df_allocation["Completion_Date"].replace({ 
    'June 24. 2024': '2024-06-01 00:00:00',  
    '11/21/2024\n7/30/2025 (Q4)': '2024-11-21 00:00:00', 
    'Jun-26': '2026-01-01 00:00:00', 
     'Jun-29': '2029-06-01 00:00:00',
    'Complete\n11/12/2019': '2019-11-12 00:00:00' , 
    'Deallocated': '', 
    'Jun-28': '2028-06-01 00:00:00',  
    'Jun-25': '2025-06-01 00:00:00', 
    'Jun-23':'2023-06-01 00:00:00', 
    'Jun-27': '2027-06-01 00:00:00',
    'Jan-25': '2025-01-01 00:00:00',
    '11-21-20247-30-2025 (Q4)':'2025-07-30 00:00:00',
    '6-30-202112-31-2021': '2021-12-31 00:00:00',
    '6-1-2019': '2019-06-01 00:00:00',
    '2-11-2018': '2018-02-11 00:00:00',
     '6-30-2020': '2020-06-30 00:00:00',
    ' 6-30-2018': '2018-06-30 00:00:00',
     '6-29-2020': '2020-06-29 00:00:00',
     '11-1-2019': '2019-11-01 00:00:00',
     ' 12-10-2018': '2018-12-10 00:00:00',
     ' 11-13-2019': '2019-11-13 00:00:00',
     '3-30-2020':'2020-03-30 00:00:00',
    ' 6-30-2020': '2020-06-30 00:00:00',
    '11-12-2019': '2019-11-12 00:00:00',
    '1-31-2020': '2020-01-31 00:00:00',
    '8-30-2020': '2020-08-30 00:00:00',
    '5-16-2020': '2020,05-16 00:00:00',
     '5-7-2020': '2020-05-07 00:00:00'}, inplace =True)

In [23]:
#cleaning up 3rd Party dates
df_allocation["Third_Party_Award_Date"].replace({ 
'Augsut 12, 2021': '2021-08-12 00:00:00',
'43435': '2018-12-01 00:00:00',
'07-29-2020': '2020-07-29 00:00:00',
'43497' : '2019-02-01 00:00:00',
'TBD 6-24-2021' : 'TBD',
'TBD 6-30-2022' : 'TBD'
}, inplace =True)

In [24]:
#coercing to dates
df_allocation = df_allocation.assign(
    Allocation_Date_New = pd.to_datetime(df_allocation.Allocation_Date, errors="coerce").dt.date,
    Third_Party_Award_Date_New = pd.to_datetime(df_allocation.Third_Party_Award_Date, errors="coerce").dt.date,
    Completion_Date_New = pd.to_datetime(df_allocation.Completion_Date, errors="coerce").dt.date
)

In [25]:
#checking for nas
df_allocation.isna().sum()

Award_Year                      0
Award_Recipient                 0
Implementing_Agency             0
Phase                           1
Allocation_Date                 0
Completion_Date                 0
Third_Party_Award_Date          0
Components                      0
PPNO                            0
PPNO_New2                     292
Allocation_Date_New            83
Third_Party_Award_Date_New    145
Completion_Date_New            97
dtype: int64

## Final CSV Version

In [26]:
#drop old columns
df_allocation = df_allocation.drop(['PPNO_New2','Allocation_Date','Completion_Date','Third_Party_Award_Date'], axis=1)
#rename columns
df_allocation = df_allocation.rename(columns = {'Allocation_Date_New':'Allocation_Date', 'Completion_Date_New':'Completion_Date', 'Third_Party_Award_Date_New':'Third_Party_Award_Date'})

In [27]:
df_allocation.head(100)

,Award_Year,Award_Recipient,Implementing_Agency,Phase,Components,PPNO,Allocation_Date,Third_Party_Award_Date,Completion_Date
0,2015,Antelope Valley Transit Authority,Antelope Valley Transit Authority,CONST,Purchase 13 60-foot articulated BRT buses and ...,CP005,2015-10-22,2016-03-14,2022-03-30
1,2015,Capitol Corridor Joint Powers Authority,Capitol Corridor Joint Powers Authority,CONST,Track and curve improvements between San Jose ...,CP012,2016-05-19,2016-06-01,2019-06-01
2,2015,Los Angeles County Metropolitan Transportation...,Los Angeles County Metropolitan Transportation...,CONST,Replace Blue Line signal system; install new t...,CP015,2016-06-30,2017-06-28,2021-06-30
3,2015,Los Angeles-San Diego-San Luis Obispo Rail Cor...,Los Angeles-San Diego-San Luis Obispo Rail Cor...,CONST,LOSSAN and 12 transit agencies from San Luis O...,CP007,2015-12-10,2016-06-01,2021-06-30
4,2015,Monterey-Salinas Transit,Monterey-Salinas Transit,CONST,Renovation and expansion of the Monterey maint...,CP013,2016-05-19,2016-11-03,2018-09-30
5,2015,Orange County Transportation Authority (OCTA),Orange County Transportation Authority (OCTA),CONST,Purchase five 40-foot CNG buses for BRT Route ...,CP004,2015-10-22,2015-11-30,2018-02-11
6,2015,Sacramento Regional Transit District,Sacramento Regional Transit District,CONST,Refurbishment of Seven UTDC Light Rail Vehicle...,CP001,2015-08-27,2015-10-21,2020-06-30
7,2015,San Diego Association of Governments (SANDAG),San Diego Association of Governments (SANDAG),CONST,Bus Rapid Transit Route between Downtown San D...,CP003,2015-08-27,2016-01-08,2020-09-30
8,2015,San Diego Metropolitan Transit System (MTS),San Diego Metropolitan Transit System (MTS),PS&E,Courthouse Trolley Station (Design),CP008,2016-01-21,NaT,2018-06-30
9,2015,San Diego Metropolitan Transit System (MTS),San Diego Metropolitan Transit System (MTS),CONST,Courthouse Trolley Station (Construction),CP008,2017-06-29,2017-05-30,2020-06-29


In [28]:
#df_allocation.to_csv("./final_df_allocation.csv", index= False)

# Cleaning Project Sheet



## Filling NA for TIRCP, Allocated, and Expended Amounts

In [29]:
df_project[['TIRCP_Award_Amount_($)', 'Allocated_Amount','Expended_Amount']] = df_project[['TIRCP_Award_Amount_($)', 'Allocated_Amount', 'Expended_Amount']].fillna(value=0)

/opt/conda/lib/python3.9/site-packages/pandas/core/frame.py:3641: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[k1] = value[k2]


## Cleaning up PPNO Numbers based on Allocation Sheet

In [30]:
#slicing PPNO to be 5 characters
df_project = df_project.assign(PPNO_New = df_project['PPNO'].str.slice(start=0, stop=5)) 

In [31]:
#importing Excel crosswalk sheet
#importing Excel crosswalk sheet
FILE_NAME3 = "Projects_PPNO.xlsx"
project_ppno = pd.read_excel(f"{GCS_FILE_PATH}{FILE_NAME3}")

In [32]:
#Merge in Crosswalk 
df_project2 = pd.merge(df_project, project_ppno, on = ["Award_Year", "Local_Agency"], how = "left")

In [33]:
#some values in PPNO and PPNO_New2 are strings, some are floats...so have to convert PPNO New 2 to strings
df_project2.PPNO_New = df_project2.apply(lambda x: x.PPNO_New if (str(x.PPNO_New2) == 'nan') else x.PPNO_New2, axis=1)

In [34]:
#making sure PPNO_New is a string 
df_project2 = df_project2.astype({'PPNO_New': 'str'})

In [35]:
PPNO_project = set(df_project2.PPNO_New.unique().tolist())
PPNO_allocation = set(df_allocation.PPNO.unique().tolist())

In [36]:
PPNO_project - PPNO_allocation #checking for differences - none. yay. 

set()

## Final CSV for project

In [37]:
df_project2[['TIRCP_Award_Amount_($)', 'Allocated_Amount','Expended_Amount']] = df_project2[['TIRCP_Award_Amount_($)', 'Allocated_Amount', 'Expended_Amount']].fillna(value=0)

In [38]:
#dividing allocated amount and expended amount
df_project2['Expended_Percent'] = df_project2['Expended_Amount']/df_project2['Allocated_Amount']
df_project2['Allocated_Percent'] = df_project2['Allocated_Amount']/df_project2['TIRCP_Award_Amount_($)']

In [39]:
#filling in percentages that are na with 0
df_project2[['Expended_Percent','Allocated_Percent']] = df_project2[['Expended_Percent','Allocated_Percent']].fillna(value=0)

In [40]:
#Categorize Expended Percent into bins
def expended_percent(row):
            if row.Expended_Percent == 0:
                return "No expenditure recorded"
            elif ((row.Expended_Percent > 0) and (row.Expended_Percent < .50)):
                return "1-50"
            elif row.Expended_Percent < 0.71:
                return "51-70"
            else:
                return "71-100"

In [41]:
df_project2["Expended_Percent_Group"] = df_project2.apply(lambda x: expended_percent(x), axis=1)

In [42]:
df_project2.dtypes

Award_Year                  int64
Project_#                   int64
Local_Agency               object
Project_Title              object
PPNO                       object
Key_Project_Elements       object
TIRCP_Award_Amount_($)    float64
Allocated_Amount            int64
Expended_Amount           float64
PPNO_New                   object
PPNO_New2                  object
Expended_Percent          float64
Allocated_Percent         float64
Expended_Percent_Group     object
dtype: object

In [43]:
# Categorize years and expended_percent_group into bins
def progress(df):   
    if (df['Award_Year'] == 2015) and (df['Expended_Percent_Group'] == "1-50"):
        return 'Behind'
    elif (df['Award_Year'] == 2015) and (df['Expended_Percent_Group'] == "51-70"):
        return 'On Track'
    elif (df['Award_Year'] == 2015) and (df['Expended_Percent_Group'] == "71-100"):
        return 'On Track'
    elif (df['Award_Year'] == 2016) and (df['Expended_Percent_Group'] == "1-50"):
        return 'Behind'
    elif (df['Award_Year'] == 2016) and (df['Expended_Percent_Group'] == "71-100"):
        return 'On Track'
    elif (df['Award_Year'] == 2016) and (df['Expended_Percent_Group'] == "51-70"):
        return 'On Track'
    elif (df['Award_Year'] == 2018) and (df['Expended_Percent_Group'] == "1-50"):
        return 'On Track'
    elif (df['Award_Year'] == 2018) and (df['Expended_Percent_Group'] == "51-70"):
        return 'Ahead'
    elif (df['Award_Year'] == 2018) and (df['Expended_Percent_Group'] == "71-100"):
        return 'Ahead'
    elif (df['Award_Year'] == 2020) and (df['Expended_Percent_Group'] == "1-50"):
        return 'On Track'
    elif (df['Award_Year'] == 2020) and (df['Expended_Percent_Group'] == "51-70"):
        return 'Ahead'
    elif (df['Award_Year'] == 2020) and (df['Expended_Percent_Group'] == "71-100"):
        return 'Ahead'
    else: 
        return "No Expenditures"

In [44]:
df_project2['Progress'] = df_project2.apply(progress, axis = 1)

In [45]:
#drop old column
df_project2 = df_project2.drop(['PPNO','PPNO_New2'], axis=1)

In [46]:
#renaming PPNO 
df_project2 = df_project2.rename(columns = {'PPNO_New':'PPNO'})

In [47]:
df_project2.shape

(74, 13)

In [48]:
df_project2.sample(4)

,Award_Year,Project_#,Local_Agency,Project_Title,Key_Project_Elements,TIRCP_Award_Amount_($),Allocated_Amount,Expended_Amount,PPNO,Expended_Percent,Allocated_Percent,Expended_Percent_Group,Progress
12,2015,13,Southern California Regional Rail Authority,Purchase of Nine Fuel-Efficient Tier IV EMD F-...,Purchase nine of 20 locomotives that complete ...,41181000.0,41181000,37583067.00,CP002,0.912631,1.000000,71-100,On Track
15,2016,2,Capitol Corridor Joint Powers Authority,"Increased Rail Service to Roseville, Service O...",Construct track and facility improvements for ...,8999000.0,3446000,532908.06,CP023,0.154645,0.382931,1-50,Behind
58,2020,2,Bay Area Rapid Transit District (BART),The Transbaby Corridor Core Capacity Program: ...,Expansion of the Core Capacity rail car fleet ...,107100000.0,0,0.00,CP060,0.000000,0.000000,No expenditure recorded,No Expenditures
57,2020,1,Antelope Valley Transit Authority (AVTA),Reaching the Most Transit-Vulnerable: AVTA's Z...,Purchase of 11 zero emission battery electric ...,6503000.0,6503000,0.00,CP059,0.000000,1.000000,No expenditure recorded,No Expenditures


In [49]:
df_project2.to_csv("gs://calitp-analytics-data/data-analyses/tircp/df_project2.csv", index = False)

# Merging Project & Allocations
Merge on PPNO & Award_Year

In [50]:
#left merge to keep everything on the alllocation side since there are more records in that dataset 
df_combined2 = df_allocation.merge(df_project2, how = "left", on = ["PPNO", "Award_Year"])

In [51]:
df_combined2.shape

(319, 20)

In [52]:
df_combined2.drop_duplicates #dropping duplicates

<bound method DataFrame.drop_duplicates of      Award_Year                                    Award_Recipient  \
0          2015                 Antelope Valley Transit Authority    
1          2015            Capitol Corridor Joint Powers Authority   
2          2015  Los Angeles County Metropolitan Transportation...   
3          2015  Los Angeles-San Diego-San Luis Obispo Rail Cor...   
4          2015                           Monterey-Salinas Transit   
..          ...                                                ...   
314        2020                    Solano Transportation Authority   
315        2020                    Solano Transportation Authority   
316        2020                        Torrance Transit Department   
317        2020   Transit Joint Powers Authority for Merced County   
318        2020  San Francisco Bay Area Water Emergency Transpo...   

                                   Implementing_Agency  Phase  \
0                   Antelope Valley Transit Authori

In [53]:
df_combined2[['TIRCP_Award_Amount_($)', 'Allocated_Amount','Expended_Amount']] = df_combined2[['TIRCP_Award_Amount_($)', 'Allocated_Amount', 'Expended_Amount']].fillna(value=0)

In [54]:
#dividing allocated amount and expended amount
df_combined2['Expended_Percent'] = df_combined2['Expended_Amount']/df_combined2['Allocated_Amount']
df_combined2['Allocated_Percent'] = df_combined2['Allocated_Amount']/df_combined2['TIRCP_Award_Amount_($)']

In [55]:
len(df_combined2) #checking rows  before deleting duplicates

319

In [56]:
df_combined2 = df_combined2.drop_duplicates()

In [57]:
#df_combined2.to_csv("gs://calitp-analytics-data/data-analyses/tircp/df_combined2.csv", index = False)